In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_storming_round.csv')
test = pd.read_csv('test_storming_round.csv')

In [10]:
previous_month = train.loc[train['year_month'] == "8/1/2024", ['new_policy_count', 'unique_customers','agent_code']].copy()


In [11]:
previous_month = previous_month.rename(columns={'new_policy_count': 'previous_new_policy_count', 'unique_customers': 'previous_unique_customers'})
previous_month = previous_month.reset_index(drop=True)
previous_month

,previous_new_policy_count,previous_unique_customers,agent_code
0,18,22,f79f587e
1,28,16,fd4ffb55
2,30,14,df2b837a
3,26,10,26142387
4,30,22,8f699df7
...,...,...,...
900,25,18,d3b44e44
901,15,18,0b1d7054
902,30,19,51321fa2
903,0,15,dd4a0d2d


In [19]:
current_month = test.loc[test['year_month'] == "9/1/2024", ['new_policy_count', 'unique_customers','agent_code']].copy()
current_month = current_month.rename(columns={'new_policy_count': 'current_new_policy_count', 'unique_customers': 'current_unique_customers'})
current_month = current_month.reset_index(drop=True)

In [20]:
current_month

,current_new_policy_count,current_unique_customers,agent_code
0,24,20,de9a845f
1,14,9,1450745b
2,34,21,5c4dd08c
3,33,15,3c7a8b50
4,15,15,3dd0c2a1
...,...,...,...
909,21,14,8f77f8e4
910,26,8,a199082b
911,34,15,9c2e3be7
912,28,13,89cb3dd5


In [21]:
# Combining the two DataFrames on 'agent_code'
combined_df = pd.merge(previous_month, current_month, on='agent_code', how='inner')
combined_df = combined_df.reset_index(drop=True)
combined_df

,previous_new_policy_count,previous_unique_customers,agent_code,current_new_policy_count,current_unique_customers
0,18,22,f79f587e,24,10
1,28,16,fd4ffb55,30,13
2,30,14,df2b837a,14,10
3,26,10,26142387,17,20
4,30,22,8f699df7,27,11
...,...,...,...,...,...
900,25,18,d3b44e44,0,7
901,15,18,0b1d7054,35,12
902,30,19,51321fa2,36,10
903,0,15,dd4a0d2d,11,12


In [ ]:
combined_df['CN'] = (combined_df['current_new_policy_count'] - combined_df['previous_new_policy_count'])/ combined_df['previous_new_policy_count']
combined_df['CU'] = (combined_df['current_unique_customers'] - combined_df['previous_unique_customers'])/ combined_df['previous_unique_customers']

In [26]:
# If CN is inf, replace is with the difference between the current and previous new policy count
for value in combined_df['CN']:
    if np.isinf(value):
        combined_df['CN'] = (combined_df['current_new_policy_count'] - combined_df['previous_new_policy_count'])

In [27]:
combined_df

,previous_new_policy_count,previous_unique_customers,agent_code,current_new_policy_count,current_unique_customers,CN,CU
0,18,22,f79f587e,24,10,6,-0.545455
1,28,16,fd4ffb55,30,13,2,-0.187500
2,30,14,df2b837a,14,10,-16,-0.285714
3,26,10,26142387,17,20,-9,1.000000
4,30,22,8f699df7,27,11,-3,-0.500000
...,...,...,...,...,...,...,...
900,25,18,d3b44e44,0,7,-25,-0.611111
901,15,18,0b1d7054,35,12,20,-0.333333
902,30,19,51321fa2,36,10,6,-0.473684
903,0,15,dd4a0d2d,11,12,11,-0.200000


In [28]:
# If both CN and CU are positive, performance is high
# If only one of them is positive, performance is medium
# If both are negative, performance is low
combined_df['performance'] = np.where((combined_df['CN'] > 0) & (combined_df['CU'] > 0), 'high',
                                        np.where((combined_df['CN'] > 0) | (combined_df['CU'] > 0), 'medium', 'low'))
combined_df

,previous_new_policy_count,previous_unique_customers,agent_code,current_new_policy_count,current_unique_customers,CN,CU,performance
0,18,22,f79f587e,24,10,6,-0.545455,medium
1,28,16,fd4ffb55,30,13,2,-0.187500,medium
2,30,14,df2b837a,14,10,-16,-0.285714,low
3,26,10,26142387,17,20,-9,1.000000,medium
4,30,22,8f699df7,27,11,-3,-0.500000,low
...,...,...,...,...,...,...,...,...
900,25,18,d3b44e44,0,7,-25,-0.611111,low
901,15,18,0b1d7054,35,12,20,-0.333333,medium
902,30,19,51321fa2,36,10,6,-0.473684,medium
903,0,15,dd4a0d2d,11,12,11,-0.200000,medium


In [29]:
combined_df.drop(columns=['current_new_policy_count', 'previous_new_policy_count', 'current_unique_customers', 'previous_unique_customers','CN','CU'], inplace=True)

In [30]:
combined_df.to_csv('agent_performance.csv', index=False)